In [1]:
def merge_two_garment_obj(garment1_path, garment2_path, output_path):
    """
    Merge two garment OBJ files into one.
    
    Args:
        garment1_path (str): Path to first garment OBJ file
        garment2_path (str): Path to second garment OBJ file  
        output_path (str): Path for the merged OBJ file
    """
    
    def parse_obj_file(file_path):
        """Parse an OBJ file and return its components"""
        vertices = []
        texture_coords = []
        faces = []
        mtllib = None
        
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                    
                parts = line.split()
                if not parts:
                    continue
                    
                if parts[0] == 'mtllib':
                    mtllib = parts[1]
                elif parts[0] == 'v':
                    # Vertex: v x y z
                    vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
                elif parts[0] == 'vt':
                    # Texture coordinate: vt u v
                    texture_coords.append([float(parts[1]), float(parts[2])])
                elif parts[0] == 'f':
                    # Face: f v1/vt1 v2/vt2 v3/vt3
                    face_vertices = []
                    for vertex_data in parts[1:]:
                        if '/' in vertex_data:
                            v_idx, vt_idx = vertex_data.split('/')
                            face_vertices.append([int(v_idx), int(vt_idx)])
                        else:
                            face_vertices.append([int(vertex_data), None])
                    faces.append(face_vertices)
        
        return {
            'vertices': vertices,
            'texture_coords': texture_coords, 
            'faces': faces,
            'mtllib': mtllib
        }
    
    def write_merged_obj(data1, data2, output_path):
        """Write merged OBJ data to file"""
        with open(output_path, 'w') as f:
            # Write material library (use first one or create combined)
            if data1['mtllib']:
                f.write(f"mtllib {data1['mtllib']}\n")
            
            # Write vertices from first garment
            for vertex in data1['vertices']:
                f.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
            
            # Write vertices from second garment
            for vertex in data2['vertices']:
                f.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
            
            # Write texture coordinates from first garment
            for vt in data1['texture_coords']:
                f.write(f"vt {vt[0]} {vt[1]}\n")
            
            # Write texture coordinates from second garment
            for vt in data2['texture_coords']:
                f.write(f"vt {vt[0]} {vt[1]}\n")
            
            # Write faces from first garment (vertex indices stay the same)
            for face in data1['faces']:
                face_str = "f"
                for vertex_data in face:
                    if vertex_data[1] is not None:
                        face_str += f" {vertex_data[0]}/{vertex_data[1]}"
                    else:
                        face_str += f" {vertex_data[0]}"
                f.write(f"{face_str}\n")
            
            # Write faces from second garment (adjust vertex indices)
            num_vertices_1 = len(data1['vertices'])
            num_texture_coords_1 = len(data1['texture_coords'])
            
            for face in data2['faces']:
                face_str = "f"
                for vertex_data in face:
                    # Adjust vertex index by adding offset from first garment
                    new_v_idx = vertex_data[0] + num_vertices_1
                    if vertex_data[1] is not None:
                        # Adjust texture coordinate index by adding offset
                        new_vt_idx = vertex_data[1] + num_texture_coords_1
                        face_str += f" {new_v_idx}/{new_vt_idx}"
                    else:
                        face_str += f" {new_v_idx}"
                f.write(f"{face_str}\n")
    
    # Parse both OBJ files
    print(f"Parsing {garment1_path}...")
    data1 = parse_obj_file(garment1_path)
    
    print(f"Parsing {garment2_path}...")
    data2 = parse_obj_file(garment2_path)
    
    # Write merged OBJ file
    print(f"Writing merged OBJ to {output_path}...")
    write_merged_obj(data1, data2, output_path)

In [2]:
garment1_path = '/data/lixinag/project/GarmentCode/simulation_output/garments_5000_0/default_body/rand_0B1T21D8NX/rand_0B1T21D8NX_sim.obj'
garment2_path = '/data/lixinag/project/GarmentCode/simulation_output/garments_5000_0/default_body/rand_0BC49T2K2O/rand_0BC49T2K2O_sim.obj'
output_path = '/data/lixinag/project/GarmentCode/A_RESULT/merged_garment.obj'

merge_two_garment_obj(
    garment1_path,
    garment2_path,
    output_path
)

Parsing /data/lixinag/project/GarmentCode/simulation_output/garments_5000_0/default_body/rand_0B1T21D8NX/rand_0B1T21D8NX_sim.obj...
Parsing /data/lixinag/project/GarmentCode/simulation_output/garments_5000_0/default_body/rand_0BC49T2K2O/rand_0BC49T2K2O_sim.obj...
Writing merged OBJ to /data/lixinag/project/GarmentCode/A_RESULT/merged_garment.obj...
